In [1]:
from splinter import Browser
import re
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
#Set up Splinter 
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless = False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 99.0.4844
[WDM] - Get LATEST driver version for 99.0.4844


[WDM] - Driver [C:\Users\Terence Lin\.wdm\drivers\chromedriver\win32\99.0.4844.51\chromedriver.exe] found in cache


In [3]:
nvidia_gpus = ['https://www.nowinstock.net/computers/videocards/nvidia/rtx3070ti/full_history.php',
               'https://www.nowinstock.net/computers/videocards/nvidia/rtx3070/full_history.php',
              'https://www.nowinstock.net/computers/videocards/nvidia/rtx3080/full_history.php',
               'https://www.nowinstock.net/computers/videocards/nvidia/rtx3080ti/full_history.php',
               'https://www.nowinstock.net/computers/videocards/nvidia/rtx3090/full_history.php',
               'https://www.nowinstock.net/computers/videocards/nvidia/rtx3060/full_history.php',
               'https://www.nowinstock.net/computers/videocards/nvidia/rtx3060ti/full_history.php'
              ]

# url = 'https://www.nowinstock.net/computers/videocards/nvidia/rtx3070/full_history.php'
# browser.visit(url)
descriptions = []
for references in nvidia_gpus:
    browser.visit(references)
    html = browser.html
    gpu_soup = soup(html,'html.parser')
    data = gpu_soup.find_all('td')

    i=0
    for link in data:
        descriptions.append(link.string)      
        

In [4]:
timestamps = []
status = []

for i in range(len(descriptions)):
    a = descriptions[i]
    if(i % 2 ==  0):
        timestamps.append(a)
    else:
        status.append(a)

In [5]:
# for gpu in status:
#     print(gpu)

print(status[1])

Amazon - GIGABYTE RTX 3070 Ti AORUS MASTER (GV-N307TAORUS M-8GD) Out of Stock 


In [6]:
pricing = []


In [7]:
for times in timestamps:
    print(times)

Mon Mar 21 2022 - 12:53 AM EST
Mon Mar 21 2022 - 12:43 AM EST
Mon Mar 21 2022 - 12:42 AM EST
Mon Mar 21 2022 - 12:30 AM EST
Mon Mar 21 2022 - 12:27 AM EST
Mon Mar 21 2022 - 12:22 AM EST
Sun Mar 20 2022 - 11:33 PM EST
Sun Mar 20 2022 - 10:27 PM EST
Sun Mar 20 2022 - 10:13 PM EST
Sun Mar 20 2022 - 10:12 PM EST
Sun Mar 20 2022 - 9:51 PM EST
Sun Mar 20 2022 - 9:43 PM EST
Sun Mar 20 2022 - 9:37 PM EST
Sun Mar 20 2022 - 9:31 PM EST
Sun Mar 20 2022 - 9:11 PM EST
Sun Mar 20 2022 - 9:01 PM EST
Sun Mar 20 2022 - 8:57 PM EST
Sun Mar 20 2022 - 8:30 PM EST
Sun Mar 20 2022 - 8:25 PM EST
Sun Mar 20 2022 - 8:21 PM EST
Sun Mar 20 2022 - 8:12 PM EST
Sun Mar 20 2022 - 7:58 PM EST
Sun Mar 20 2022 - 7:51 PM EST
Sun Mar 20 2022 - 7:35 PM EST
Sun Mar 20 2022 - 7:27 PM EST
Sun Mar 20 2022 - 7:06 PM EST
Sun Mar 20 2022 - 4:30 PM EST
Sun Mar 20 2022 - 4:25 PM EST
Sun Mar 20 2022 - 4:24 PM EST
Sun Mar 20 2022 - 4:08 PM EST
Sun Mar 20 2022 - 3:52 PM EST
Sun Mar 20 2022 - 3:40 PM EST
Sun Mar 20 2022 - 3:36 PM EST


Sun Mar 06 2022 - 8:57 AM EST
Sun Mar 06 2022 - 3:01 AM EST
Sun Mar 06 2022 - 2:56 AM EST
Sun Mar 06 2022 - 1:46 AM EST
Sun Mar 06 2022 - 1:41 AM EST
Sun Mar 06 2022 - 1:26 AM EST
Sun Mar 06 2022 - 1:17 AM EST
Sun Mar 06 2022 - 12:37 AM EST
Sun Mar 06 2022 - 12:32 AM EST
Sun Mar 06 2022 - 12:13 AM EST
Sun Mar 06 2022 - 12:02 AM EST
Sat Mar 05 2022 - 11:15 PM EST
Sat Mar 05 2022 - 11:13 PM EST
Sat Mar 05 2022 - 11:12 PM EST
Sat Mar 05 2022 - 11:07 PM EST
Sat Mar 05 2022 - 11:06 PM EST
Sat Mar 05 2022 - 11:02 PM EST
Sat Mar 05 2022 - 8:53 PM EST
Sat Mar 05 2022 - 8:48 PM EST
Sat Mar 05 2022 - 10:34 AM EST
Sat Mar 05 2022 - 10:27 AM EST
Fri Mar 04 2022 - 10:37 PM EST
Fri Mar 04 2022 - 10:32 PM EST
Wed Mar 02 2022 - 10:36 PM EST
Wed Mar 02 2022 - 10:30 PM EST
Wed Mar 02 2022 - 10:20 PM EST
Wed Mar 02 2022 - 9:46 PM EST
Wed Mar 02 2022 - 5:20 PM EST
Wed Mar 02 2022 - 5:15 PM EST
Wed Mar 02 2022 - 9:34 AM EST
Wed Mar 02 2022 - 9:30 AM EST
Mon Feb 28 2022 - 9:36 PM EST
Mon Feb 28 2022 - 9:30 